In [1]:
import tensorflow as tf

from tensorflow.keras.utils import normalize
import os
import glob
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [2]:
SIZE_X, SIZE_Y = 288, 288
n_classes = 4  # Adjust the number of classes based on your data
IMG_SIZE = (SIZE_X, SIZE_Y)
BATCH_SIZE = 16  # Reduced batch size for better memory management

In [ ]:
# Model I

In [4]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda

def multi_unet_model(n_classes=4, IMG_HEIGHT=288, IMG_WIDTH=288, IMG_CHANNELS=1):
    # Build the model
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    # Inputs are already normalized in the preprocessing step
    s = inputs

    # Contraction path
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)
    
    # Additional contraction and expansion layers as defined in your code
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)
     
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)
     
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)
     
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = Dropout(0.3)(c5)
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)
    
    #Expansive path 
    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
     
    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
     
    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = Dropout(0.1)(c8)
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)

    # Expansive path (shortened for brevity)
    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)  # Ensure the axis is correctly set for concatenation
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = Dropout(0.1)(c9)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
    
    # Output layer with softmax for multi-class segmentation
    outputs = Conv2D(n_classes, (1, 1), activation='softmax')(c9)
    
    model = Model(inputs=[inputs], outputs=[outputs])
    return model


In [ ]:
#Model II

In [5]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, Dropout, SeparableConv2D, Activation, BatchNormalization

def multi_unet_model(n_classes=4, IMG_HEIGHT=288, IMG_WIDTH=288, IMG_CHANNELS=1):
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    
    def conv_block(inputs, num_filters):
        x = SeparableConv2D(num_filters, 3, padding="same", 
                            depthwise_initializer="he_normal",
                            pointwise_initializer="he_normal")(inputs)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        x = SeparableConv2D(num_filters, 3, padding="same", 
                            depthwise_initializer="he_normal",
                            pointwise_initializer="he_normal")(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        return x
    
    def encoder_block(inputs, num_filters):
        x = conv_block(inputs, num_filters)
        p = MaxPooling2D((2, 2))(x)
        p = Dropout(0.1)(p)
        return x, p
    
    def decoder_block(inputs, skip_features, num_filters):
        x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
        x = concatenate([x, skip_features])
        x = conv_block(x, num_filters)
        return x

    # Contraction path
    c1, p1 = encoder_block(inputs, 16)
    c2, p2 = encoder_block(p1, 32)
    c3, p3 = encoder_block(p2, 64)
    c4, p4 = encoder_block(p3, 128)
    c5 = conv_block(p4, 256)

    # Expansive path
    u6 = decoder_block(c5, c4, 128)
    u7 = decoder_block(u6, c3, 64)
    u8 = decoder_block(u7, c2, 32)
    u9 = decoder_block(u8, c1, 16)

    outputs = Conv2D(n_classes, (1, 1), activation='softmax')(u9)
    
    model = Model(inputs=[inputs], outputs=[outputs])
    return model


In [8]:
# Import TensorFlow and required components
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, Dropout

# Define paths and constants
train_image_dir = "/Users/arahjou/Downloads/dataset_UWM_GI_Tract_train_valid/train/images/*.png"
train_mask_dir = "/Users/arahjou/Downloads/dataset_UWM_GI_Tract_train_valid/train/masks/*.png"
IMG_SIZE = (288, 288)  # Assuming this is your desired image size
BATCH_SIZE = 16
n_classes = 4  # Number of classes for segmentation

# Load and preprocess images and masks
def load_image_and_mask(image_path, mask_path):
    def decode_img(img_path):
        img = tf.io.read_file(img_path)
        img = tf.image.decode_png(img, channels=1)
        return tf.image.resize(img, IMG_SIZE)

    image = decode_img(image_path)
    mask = decode_img(mask_path)
    mask = tf.cast(mask, tf.int8)
    mask = tf.one_hot(tf.squeeze(mask, axis=-1), depth=n_classes)
    return image, mask

# Function to normalize images and expand dimensions
def preprocess(image, mask):
    image = tf.cast(image, tf.float16) / 255.0
    return image, mask

# Dataset creation
image_paths = tf.data.Dataset.list_files(train_image_dir, seed=42)
mask_paths = tf.data.Dataset.list_files(train_mask_dir, seed=42)
dataset = tf.data.Dataset.zip((image_paths, mask_paths))
dataset = dataset.map(load_image_and_mask, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.shuffle(1000)

# Calculate dataset sizes
total_size = dataset.cardinality().numpy()
val_size = total_size // 10
test_size = total_size // 10
train_size = total_size - (val_size + test_size)

# Dataset partitioning
train_dataset = dataset.take(train_size).batch(BATCH_SIZE, drop_remainder=True)
val_dataset = dataset.skip(train_size).take(val_size).batch(BATCH_SIZE, drop_remainder=True)
test_dataset = dataset.skip(train_size + val_size).batch(BATCH_SIZE, drop_remainder=True)

# Prefetching datasets
train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset = val_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

# Model creation function using the detailed U-Net architecture
def get_model(n_classes, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS):
    return multi_unet_model(n_classes=n_classes, IMG_HEIGHT=IMG_HEIGHT, IMG_WIDTH=IMG_WIDTH, IMG_CHANNELS=IMG_CHANNELS)

# Setup mixed precision policy
from tensorflow.keras import mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

# Create and compile the model
model = get_model(n_classes, IMG_SIZE[0], IMG_SIZE[1], 1)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['Accuracy'])

# Training the model
history = model.fit(
    train_dataset,
    epochs=3,
    validation_data=val_dataset,
    verbose=1
)

# Save the trained model
model.save('optimized_model_3epochs_multi_U_net.keras')


Epoch 1/3
662/662 ━━━━━━━━━━━━━━━━━━━━ 457s 679ms/step - Accuracy: 0.8165 - loss: 0.6677 - val_Accuracy: 0.9619 - val_loss: 0.1520
Epoch 2/3
662/662 ━━━━━━━━━━━━━━━━━━━━ 446s 673ms/step - Accuracy: 0.9712 - loss: 0.0892 - val_Accuracy: 0.9772 - val_loss: 0.0686
Epoch 3/3
662/662 ━━━━━━━━━━━━━━━━━━━━ 678s 1s/step - Accuracy: 0.9795 - loss: 0.0604 - val_Accuracy: 0.9808 - val_loss: 0.0581


Dealing with class imbalance and preventing overfitting are two important aspects of training a neural network, especially for tasks like segmentation where certain classes may be underrepresented. Here are strategies to handle both issues effectively:

### Handling Class Imbalance

1. **Weighted Loss Function:**
   You can assign a higher loss weight to underrepresented classes. TensorFlow allows you to use `class_weight` in the `fit` method for classification tasks. For segmentation, you can modify your loss function directly to account for class imbalance by using weights inside the categorical cross-entropy calculation.

   ```python
   from tensorflow.keras import backend as K

   def weighted_categorical_crossentropy(weights):
       def wcce(y_true, y_pred):
           Kweights = K.constant(weights)
           y_true = K.cast(y_true, y_pred.dtype)
           return K.categorical_crossentropy(y_true, y_pred) * K.sum(y_true * Kweights, axis=-1)
       return wcce

   # Example of weights for three classes where the second class is more rare
   weights = [1, 2, 1]
   loss = weighted_categorical_crossentropy(weights)
   ```

2. **Focal Loss:**
   This is particularly useful for handling class imbalance in segmentation. It modifies the cross-entropy loss to down-weight easy examples and focus training on hard negatives.

   ```python
   def focal_loss(gamma=2., alpha=4.):
       gamma = float(gamma)
       alpha = float(alpha)
       def focal_loss_fixed(y_true, y_pred):
           epsilon = K.epsilon()
           y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
           y_true = K.cast(y_true, y_pred.dtype)
           alpha_t = y_true*alpha + (K.ones_like(y_true)-y_true)*(1-alpha)
           p_t = y_true*y_pred + (K.ones_like(y_true)-y_true)*(1-y_pred)
           fl = - K.log(p_t) * K.pow((K.ones_like(y_true)-p_t), gamma) * alpha_t
           return K.mean(fl)
       return focal_loss_fixed
   ```

### Preventing Overfitting

1. **Regularization:**
   Adding L1 or L2 regularization to the convolutional layers can help reduce overfitting by penalizing large weights.

   ```python
   from tensorflow.keras.regularizers import l2

   # Example adding L2 regularization to a layer
   x = Conv2D(filters, (3, 3), padding='same', kernel_regularizer=l2(0.01))(x)
   ```

2. **Dropout:**
   You're already using Dropout, which is great. You might experiment with placing Dropout layers at different points in your network or adjusting the dropout rate.

3. **Early Stopping:**
   Monitor validation loss and stop training when it starts to increase, even if the training loss continues to decrease.

   ```python
   from tensorflow.keras.callbacks import EarlyStopping

   early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
   ```

4. **Data Augmentation:**
   Increasing the diversity of your training set can help generalize better. You can use Keras' `ImageDataGenerator` or `tf.image` for real-time data augmentation.

   ```python
   from tensorflow.keras.preprocessing.image import ImageDataGenerator

   datagen = ImageDataGenerator(
       rotation_range=20,
       width_shift_range=0.2,
       height_shift_range=0.2,
       shear_range=0.1,
       zoom_range=0.2,
       horizontal_flip=True,
       fill_mode='nearest'
   )
   ```

### Compile and Fit the Model

Incorporate these strategies when compiling and fitting your model:

```python
model.compile(optimizer='adam', loss=focal_loss(), metrics=['accuracy'])

# Assuming 'train_data' and 'val_data' are your training and validation datasets
model.fit(train_data, epochs=50, validation_data=val_data, callbacks=[early_stopping])
```

Adjust these strategies based on the specific requirements and data characteristics of your project. Each technique can have a significant impact, so it’s worth experimenting with various combinations to see what works best for your specific scenario.

In [9]:
import tensorflow as tf
import numpy as np
from keras.metrics import MeanIoU

# Prepare to collect predictions and actuals
y_pred_list = []
y_true_list = []

# Iterate over the test dataset
for x_batch, y_batch in test_dataset:
    y_pred = model.predict(x_batch)
    y_pred_argmax = np.argmax(y_pred, axis=3)
    y_true_list.append(y_batch.numpy()[:, :, :, 0])
    y_pred_list.append(y_pred_argmax)

# Convert lists to single numpy arrays
y_true = np.concatenate(y_true_list, axis=0)
y_pred_argmax = np.concatenate(y_pred_list, axis=0)

# Calculate Mean IoU using Keras
n_classes = 4
IOU_keras = MeanIoU(num_classes=n_classes)
IOU_keras.update_state(y_true, y_pred_argmax)
print("Mean IoU =", IOU_keras.result().numpy())
confusion_mtx = IOU_keras.total_cm.numpy()  # Accessing the confusion matrix
print(confusion_mtx)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━

2024-04-29 23:04:14.926753: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Mean IoU = 0.01840406
[[ 1317112.   795464.  1227881.   944340.]
 [16777216.   198817.   119481.    50731.]
 [       0.        0.        0.        0.]
 [       0.        0.        0.        0.]]


In [10]:
import matplotlib.pyplot as plt

# Extract a single batch from the dataset
for x_batch, y_batch in test_dataset.take(1):
    test_img = x_batch[0]  # Take the first image from the batch
    ground_truth = y_batch[0]  # Corresponding ground truth
    test_img_input = tf.expand_dims(test_img[:, :, 0], axis=0)  # Ensure dimensions are correct

    # Make a prediction
    prediction = model.predict(test_img_input)
    predicted_img = np.argmax(prediction, axis=3)[0, :, :]  # Convert predictions to label format

    # Plotting
    plt.figure(figsize=(12, 8))
    plt.subplot(231)
    plt.title('Testing Image')
    plt.imshow(test_img[:, :, 0], cmap='gray')  # Display the first channel
    plt.subplot(232)
    plt.title('Testing Label')
    plt.imshow(ground_truth[:, :, 0], cmap='jet')  # Display the first channel of ground truth
    plt.subplot(233)
    plt.title('Prediction on test image')
    plt.imshow(predicted_img, cmap='jet')
    plt.show()


ValueError: as_list() is not defined on an unknown TensorShape.

In [11]:
# Class-wise IoU from confusion matrix
class_iou = []
for i in range(n_classes):
    iou = confusion_mtx[i, i] / (np.sum(confusion_mtx[i, :]) + np.sum(confusion_mtx[:, i]) - confusion_mtx[i, i])
    class_iou.append(iou)
    print(f"IoU for class {i+1} is: {iou}")


IoU for class 1 is: 0.06253495812416077
IoU for class 2 is: 0.011081274598836899
IoU for class 3 is: 0.0
IoU for class 4 is: 0.0


In [12]:
import numpy as np
import matplotlib.pyplot as plt

def label_to_color_image(label):
    """Convert a 2D array label to a color image.
    
    Args:
        label: A 2D array with integer type, storing the segmentation label.
    
    Returns:
        result: A 2D array with three channels (RGB), where each element in the label
                is mapped to a corresponding RGB color.
    """
    # Define the colormap
    color_map = np.array([
        [0, 0, 0],        # Class 0 -> Black
        [255, 0, 0],      # Class 1 -> Red
        [0, 255, 0],      # Class 2 -> Green
        [0, 0, 255]       # Class 3 -> Blue
    ])

    # Map the label to the corresponding color
    img = np.take(color_map, label, axis=0)

    return img

# Example usage:
# Assuming 'predicted_img' is a 2D array with class labels as integers (output from argmax)
# predicted_img_color = label_to_color_image(predicted_img)
# plt.imshow(predicted_img_color)


In [13]:
for x_batch, y_batch in test_dataset.take(1):
    test_img = x_batch[0]  # First image in the batch
    ground_truth = y_batch[0]  # Corresponding ground truth
    test_img_input = tf.expand_dims(test_img[:, :, 0], axis=0)  # Prepare input

    prediction = model.predict(test_img_input)
    predicted_img = np.argmax(prediction, axis=3)[0, :, :]  # Convert predictions to label format
    predicted_img_color = label_to_color_image(predicted_img)  # Convert to color

    # Plotting
    plt.figure(figsize=(12, 8))
    plt.subplot(231)
    plt.title('Testing Image')
    plt.imshow(test_img[:, :, 0], cmap='gray')
    plt.subplot(232)
    plt.title('Testing Label')
    plt.imshow(label_to_color_image(ground_truth[:, :, 0]))  # Display ground truth in color
    plt.subplot(233)
    plt.title('Prediction on test image')
    plt.imshow(predicted_img_color)
    plt.show()


ValueError: as_list() is not defined on an unknown TensorShape.